In [142]:
import pandas as pd
import numpy as np
import json
import time
import re
import requests
from bs4 import BeautifulSoup
import itertools
import datetime

In [2]:
completed = 'http://ufcstats.com/statistics/events/completed?page=all'
url_request = requests.get(completed).text
soup = BeautifulSoup(url_request, 'html.parser')

In [3]:
events_data = []
links_data = []
table = soup.find('table',{'class':'b-statistics__table-events'})
table_rows = table.find_all('tr')
for row in table_rows[3:]:
    events_data.append([t.text.strip() for t in row.find_all('td')]) 
    links_data.append(row.find('a')['href'])

In [4]:
events_df = pd.DataFrame(events_data, columns=['Name-Date', 'Location'])
events_df['url'] = links_data
events_df

,Name-Date,Location,url
0,UFC Fight Night: Overeem vs. Harris\n ...,"Jacksonville, Florida, USA",http://ufcstats.com/event-details/dfb965c98244...
1,UFC Fight Night: Smith vs. Teixeira\n ...,"Jacksonville, Florida, USA",http://ufcstats.com/event-details/5f8e00c27b7e...
2,UFC 249: Ferguson vs. Gaethje\n ...,"Jacksonville, Florida, USA",http://ufcstats.com/event-details/898337ef520f...
3,UFC Fight Night: Lee vs. Oliveira\n ...,"Brasilia, Distrito Federal, Brazil",http://ufcstats.com/event-details/53278852bcd9...
4,UFC 248: Adesanya vs. Romero\n ...,"Las Vegas, Nevada, USA",http://ufcstats.com/event-details/0b5b6876c2a4...
...,...,...,...
511,UFC 5: The Return of the Beast\n ...,"Charlotte, North Carolina, USA",http://ufcstats.com/event-details/dedc3bb440d0...
512,UFC 4: Revenge of the Warriors\n ...,"Tulsa, Oklahoma, USA",http://ufcstats.com/event-details/b60391da771d...
513,UFC 3: The American Dream\n ...,"Charlotte, North Carolina, USA",http://ufcstats.com/event-details/1a49e0670dfa...
514,UFC 2: No Way Out\n \n\...,"Denver, Colorado, USA",http://ufcstats.com/event-details/a6a9ab5a824e...


In [127]:
events_df['Event'] = events_df['Name-Date'].str.split("\n",expand=True)[0].str.strip()
events_df['Date'] = events_df['Name-Date'].str.split("\n",expand=True)[3].str.strip()

In [130]:
events_df = events_df[['Date', 'Event', 'Location', 'url']]

In [131]:
events_df.to_csv("data_ufcstats/ufc-stats-events-list.csv", index=False)

In [144]:
events_df = pd.read_csv("data_ufcstats/ufc-stats-events-list.csv")
events_df.head()

,Date,Event,Location,url
0,"May 16, 2020",UFC Fight Night: Overeem vs. Harris,"Jacksonville, Florida, USA",http://ufcstats.com/event-details/dfb965c98244...
1,"May 13, 2020",UFC Fight Night: Smith vs. Teixeira,"Jacksonville, Florida, USA",http://ufcstats.com/event-details/5f8e00c27b7e...
2,"May 09, 2020",UFC 249: Ferguson vs. Gaethje,"Jacksonville, Florida, USA",http://ufcstats.com/event-details/898337ef520f...
3,"March 14, 2020",UFC Fight Night: Lee vs. Oliveira,"Brasilia, Distrito Federal, Brazil",http://ufcstats.com/event-details/53278852bcd9...
4,"March 07, 2020",UFC 248: Adesanya vs. Romero,"Las Vegas, Nevada, USA",http://ufcstats.com/event-details/0b5b6876c2a4...


In [145]:
# Get event details from each event
matches_df = pd.DataFrame()
for index, row in events_df.iterrows():
    event_name = row['Event']
    event_url = row['url']
    event_date = row['Date']
    print(str(index) + "\t" + event_date + "\t" + event_name)
    print(str(event_url))
    try:
        url_request = requests.get(event_url).text
        soup = BeautifulSoup(url_request, 'html.parser')
        table_rows = soup.find_all('tr',{'class':'b-fight-details__table-row'})
        data = []
        for row in table_rows[1:]:
            data.append([t.text.strip() for t in row.find_all('td')]) 
        df = pd.DataFrame(data, columns=['Results', 'Fighters', 'STR', 'TD', 'SUB', 'PASS', 'WEIGHT CLASS', 'METHOD', 'ROUND', 'TIME'])
        df.insert(0, "Date", event_date) 
        df.insert(1, "Event", event_name)
    except:
        print("Error")
    matches_df = matches_df.append(df, ignore_index=True)


0	May 16, 2020	UFC Fight Night: Overeem vs. Harris
http://ufcstats.com/event-details/dfb965c9824425db
1	May 13, 2020	UFC Fight Night: Smith vs. Teixeira
http://ufcstats.com/event-details/5f8e00c27b7e7410
2	May 09, 2020	UFC 249: Ferguson vs. Gaethje
http://ufcstats.com/event-details/898337ef520fe4d3
3	March 14, 2020	UFC Fight Night: Lee vs. Oliveira
http://ufcstats.com/event-details/53278852bcd91e11
4	March 07, 2020	UFC 248: Adesanya vs. Romero
http://ufcstats.com/event-details/0b5b6876c2a4723f
5	February 29, 2020	UFC Fight Night: Benavidez vs. Figueiredo
http://ufcstats.com/event-details/fc9a9559a05f2704
6	February 22, 2020	UFC Fight Night: Felder vs. Hooker
http://ufcstats.com/event-details/33b2f68ef95252e0
7	February 15, 2020	UFC Fight Night: Anderson vs. Blachowicz
http://ufcstats.com/event-details/5df17b3620145578
8	February 08, 2020	UFC 247: Jones vs. Reyes
http://ufcstats.com/event-details/b26d3e3746fb4024
9	January 25, 2020	UFC Fight Night: Blaydes vs. Dos Santos
http://ufcstats

78	May 19, 2018	UFC Fight Night: Maia vs. Usman
http://ufcstats.com/event-details/3144121470023e9a
79	May 12, 2018	UFC 224: Nunes vs. Pennington
http://ufcstats.com/event-details/67ec58d7cf599835
80	April 21, 2018	UFC Fight Night: Barboza vs. Lee
http://ufcstats.com/event-details/ad99fa5325519169
81	April 14, 2018	UFC Fight Night: Poirier vs. Gaethje
http://ufcstats.com/event-details/620be7e0712d431b
82	April 07, 2018	UFC 223: Khabib vs. Iaquinta
http://ufcstats.com/event-details/49e7e05f902479ca
83	March 17, 2018	UFC Fight Night: Werdum vs. Volkov
http://ufcstats.com/event-details/eed2b71d77d95416
84	March 03, 2018	UFC 222: Cyborg vs. Kunitskaya
http://ufcstats.com/event-details/7929be8290289a47
85	February 24, 2018	UFC Fight Night: Emmett vs. Stephens
http://ufcstats.com/event-details/daa89f01e1c0f42a
86	February 18, 2018	UFC Fight Night: Cerrone vs. Medeiros
http://ufcstats.com/event-details/d181689653115b6a
87	February 10, 2018	UFC 221: Romero vs. Rockhold
http://ufcstats.com/event

156	June 18, 2016	UFC Fight Night: MacDonald vs Thompson
http://ufcstats.com/event-details/563d051c9e769b24
157	June 04, 2016	UFC 199: Rockhold vs. Bisping 2
http://ufcstats.com/event-details/ff4c3ab594c7fac3
158	May 29, 2016	UFC Fight Night: Almeida vs Garbrandt
http://ufcstats.com/event-details/990060b2a68a7b82
159	May 14, 2016	UFC 198: Werdum vs Miocic
http://ufcstats.com/event-details/a4bf17bd3ba3423b
160	May 08, 2016	UFC Fight Night: Overeem vs Arlovski
http://ufcstats.com/event-details/a314687f372b2cec
161	April 23, 2016	UFC 197: Jones vs Saint Preux
http://ufcstats.com/event-details/3d481aa374c954a1
162	April 16, 2016	UFC on FOX: Teixeira vs Evans
http://ufcstats.com/event-details/e7e970d508529bf3
163	April 10, 2016	UFC Fight Night: Rothwell vs Dos Santos
http://ufcstats.com/event-details/4887e5bc4dbb73ff
164	March 19, 2016	UFC Fight Night: Hunt vs Mir
http://ufcstats.com/event-details/e60201cfab7d656d
165	March 05, 2016	UFC 196: McGregor vs Diaz
http://ufcstats.com/event-detail

234	July 16, 2014	UFC Fight Night: Cowboy vs Miller
http://ufcstats.com/event-details/79ded75550efc139
235	July 06, 2014	The Ultimate Fighter: Team Edgar vs. Team Penn Finale
http://ufcstats.com/event-details/1c5879330d42255f
236	July 05, 2014	UFC 175: Weidman vs Machida
http://ufcstats.com/event-details/53adf5b845d91e4a
237	June 28, 2014	UFC Fight Night: Swanson vs Stephens
http://ufcstats.com/event-details/c0ed7b208197e8de
238	June 28, 2014	UFC Fight Night: Te Huna vs Marquardt
http://ufcstats.com/event-details/ac9521250dc1a14c
239	June 14, 2014	UFC 174: Johnson vs Bagautinov
http://ufcstats.com/event-details/5b5307450405abf0
240	June 07, 2014	UFC Fight Night: Henderson vs Khabilov
http://ufcstats.com/event-details/1fcfc3709fe58151
241	May 31, 2014	UFC Fight Night: Miocic vs Maldonado
http://ufcstats.com/event-details/59aaf2730b84698a
242	May 31, 2014	UFC Fight Night: Munoz vs Mousasi
http://ufcstats.com/event-details/b757c73f443d4fca
243	May 24, 2014	UFC 173: Barao vs Dillashaw
http

313	May 05, 2012	UFC on FOX: Diaz vs Miller
http://ufcstats.com/event-details/5898357a45a73674
314	April 21, 2012	UFC 145: Jones vs Evans
http://ufcstats.com/event-details/155bfc7ed36622df
315	April 14, 2012	UFC on FUEL TV: Gustafsson vs Silva
http://ufcstats.com/event-details/21f2974fd08085e3
316	March 02, 2012	UFC on FX: Alves vs Kampmann
http://ufcstats.com/event-details/b5ea750025697880
317	February 25, 2012	UFC 144: Edgar vs Henderson
http://ufcstats.com/event-details/8377c5572cb356f3
318	February 15, 2012	UFC on FUEL TV: Sanchez vs Ellenberger
http://ufcstats.com/event-details/0a97691039c4bbfb
319	February 04, 2012	UFC 143: Diaz vs Condit
http://ufcstats.com/event-details/df2cf66d8c0123db
320	January 28, 2012	UFC on FOX: Evans vs Davis
http://ufcstats.com/event-details/c3c23c99477c041b
321	January 20, 2012	UFC on FX: Guillard vs Miller
http://ufcstats.com/event-details/4d74641fac830182
322	January 14, 2012	UFC 142: Aldo vs Mendes
http://ufcstats.com/event-details/18524b46c570730b

395	December 13, 2008	The Ultimate Fighter: Team Nogueira vs. Team Mir Finale
http://ufcstats.com/event-details/ea398c802d9998ee
396	December 10, 2008	UFC Fight Night - Fight for the Troops
http://ufcstats.com/event-details/30cd319d39ee689b
397	November 15, 2008	UFC 91: Couture vs Lesnar
http://ufcstats.com/event-details/a2b06ca02bca14c0
398	October 25, 2008	UFC 90: Silva vs Cote
http://ufcstats.com/event-details/c670aa48827d6be6
399	October 18, 2008	UFC 89: Bisping vs Leben
http://ufcstats.com/event-details/312f47c3d2f83ffa
400	September 17, 2008	UFC Fight Night: Diaz vs Neer
http://ufcstats.com/event-details/279093302a6f44b3
401	September 06, 2008	UFC 88: Breakthrough
http://ufcstats.com/event-details/0db70ca89e1c7374
402	August 09, 2008	UFC 87: Seek And Destroy
http://ufcstats.com/event-details/19ffeb5e3fffd6d5
403	July 19, 2008	UFC: Silva vs Irvin
http://ufcstats.com/event-details/46f11d15c0134fe3
404	July 05, 2008	UFC 86: Jackson vs Griffin
http://ufcstats.com/event-details/bf12ac

479	September 28, 2001	UFC 33: Victory in Vegas
http://ufcstats.com/event-details/c75b99887c8c3f5a
480	June 29, 2001	UFC 32: Showdown in the Meadowlands
http://ufcstats.com/event-details/7b9aa973e5c04624
481	May 04, 2001	UFC 31: Locked and Loaded
http://ufcstats.com/event-details/20bd6c3e03c46ee6
482	February 23, 2001	UFC 30: Battle on the Boardwalk
http://ufcstats.com/event-details/2b1587a3376ab743
483	December 16, 2000	UFC 29: Defense of the Belts
http://ufcstats.com/event-details/08ae5cd9aef7ddd3
484	November 17, 2000	UFC 28: High Stakes
http://ufcstats.com/event-details/da6dfd09cca1d705
485	September 22, 2000	UFC 27: Ultimate Bad Boyz
http://ufcstats.com/event-details/31652c9267606d54
486	June 09, 2000	UFC 26: Ultimate Field Of Dreams
http://ufcstats.com/event-details/cedfdf8d423d500c
487	April 14, 2000	UFC 25: Ultimate Japan 3
http://ufcstats.com/event-details/d2b1c1317a39f6c6
488	March 10, 2000	UFC 24: First Defense
http://ufcstats.com/event-details/20ec0061400178ca
489	November 

In [27]:
# page = 'http://ufcstats.com/event-details/53278852bcd91e11'
# url_request = requests.get(page).text
# soup = BeautifulSoup(url_request, 'html.parser')
# rows = soup.find_all('tr',{'class':'b-fight-details__table-row'})
# matches_data = []
# for row in rows:
#     matches_data.append([t.text.strip() for t in row.find_all('td')]) 
# matches_data_df = pd.DataFrame(matches_data, columns=['Results', 'Fighters', 'STR', 'TD', 'SUB', 'PASS', 'WEIGHT CLASS', 'METHOD', 'ROUND', 'TIME'])

In [146]:
matches_df = matches_df[matches_df['Results'].notna()]
matches_df = matches_df.drop_duplicates()
matches_df['Date']= pd.to_datetime(matches_df['Date'].str.strip(),format= '%B %d, %Y')

In [147]:
print("Total number of matches: " + str(len(matches_df)))
print("Total number of events: " +  str(len(matches_df['Event'].unique())))
print("Total number of dates: " +  str(len(matches_df['Date'].unique())))

Total number of matches: 5596
Total number of events: 516
Total number of dates: 511


In [148]:
matches_df['Results_1'] = matches_df['Results'].str.split("\n",expand=True)[0]
matches_df['Results_2'] = matches_df['Results'].str.split("\n",expand=True)[3]
matches_df['Fighter_1'] = matches_df['Fighters'].str.split("\n",expand=True)[0]
matches_df['Fighter_2'] = matches_df['Fighters'].str.split("\n",expand=True)[5]
matches_df['STR_1'] = matches_df['STR'].str.split("\n",expand=True)[0]
matches_df['STR_2'] = matches_df['STR'].str.split("\n",expand=True)[6]
matches_df['TD_1'] = matches_df['TD'].str.split("\n",expand=True)[0]
matches_df['TD_2'] = matches_df['TD'].str.split("\n",expand=True)[4]
matches_df['SUB_1'] = matches_df['SUB'].str.split("\n",expand=True)[0]
matches_df['SUB_2'] = matches_df['SUB'].str.split("\n",expand=True)[4]
matches_df['PASS_1'] = matches_df['PASS'].str.split("\n",expand=True)[0]
matches_df['PASS_2'] = matches_df['PASS'].str.split("\n",expand=True)[4]
matches_df['METHOD_result'] = matches_df['METHOD'].str.split("\n",expand=True)[0]
matches_df['METHOD_note'] = matches_df['METHOD'].str.split("\n",expand=True)[4]

In [149]:
matches_df = matches_df[['Date', 'Event', 'ROUND', 'TIME', 'Results_1', 'Results_2',
       'Fighter_1', 'Fighter_2', 'WEIGHT CLASS', 'STR_1', 'STR_2', 'TD_1', 'TD_2', 'SUB_1',
       'SUB_2', 'PASS_1', 'PASS_2', 'METHOD_result', 'METHOD_note']]

In [150]:
matches_df = matches_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [151]:
matches_df.to_csv("data_ufcstats/ufc-stats-matches-all.csv", index=False)
matches_df.columns

Index(['Date', 'Event', 'ROUND', 'TIME', 'Results_1', 'Results_2', 'Fighter_1',
       'Fighter_2', 'WEIGHT CLASS', 'STR_1', 'STR_2', 'TD_1', 'TD_2', 'SUB_1',
       'SUB_2', 'PASS_1', 'PASS_2', 'METHOD_result', 'METHOD_note'],
      dtype='object')

In [152]:
matches_subset_df = matches_df[matches_df['Date'] >='2000-11-17']
print("Number of matches: " + str(len(matches_subset_df)))
print("Number of events: " + str(len(matches_subset_df['Event'].unique())))
matches_subset_df.to_csv("data_ufcstats/ufc-stats-matches-u.csv", index=False)

Number of matches: 5335
Number of events: 485
